In [1]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objs as go
# コールバックに必要なライブラリのインポート
from dash.dependencies import Input, Output, State
# 画像利用時に必要なライブラリのインポート
from PIL import Image

/var/folders/j_/5wplb5g14fs23wsqr2cl92yc0000gn/T/ipykernel_8992/1156948000.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/j_/5wplb5g14fs23wsqr2cl92yc0000gn/T/ipykernel_8992/1156948000.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


## データの読み込み
* df_pref : 48都道府県のTOP50店をランキング形式でデータフレーム化.  df_kindのジャンルレコードと店名（nameレコード）が一致した場合、順位を新しいカラムとして追加する
* df_pref_of_parking : 都道府県ごとに駐車場の有無についてカウントしたデータフレーム
* df_kind : rankレコードを追加して、順位をレコードとして扱えるようにする


In [2]:
df_pref = pd.read_csv('../data/cleaning_pref.csv')
display(df_pref.head(3))

df_pref_of_parking = pd.read_csv('../data/groupby_parking.csv')
display(df_pref_of_parking.head())

df_kind = pd.read_csv('../data/cleaning_kind_of_rank.csv')
display(df_kind.head(3))

df_kind_of_pref = pd.read_csv('../data/groupby_kind.csv')
display(df_kind_of_pref.head(3))

,name,point,pref,address,latitude,longitude,parking,holiday,mon_holiday,tue_holiday,wed_holiday,thu_holiday,fri_holiday,sat_holiday,sun_holiday
0,麺屋 彩未,96.711,北海道,〒062-0010 北海道札幌市豊平区美園十条5丁目3-12,43.035251,141.381909,駐車場あり,月曜、火曜水曜の夜、月2回不定休有,1,1,1,0,0,0,0
1,ラーメン二郎 札幌店,95.285,北海道,〒060-0806 北海道札幌市北区北六条西8-8-11,43.067395,141.343821,駐車場なし,日曜、祝日不定休,0,0,0,0,0,0,1
2,Japanese Ramen Noodle Lab Q,94.365,北海道,〒060-0001 北海道札幌市中央区北1条西2丁目1-3 りんどうビル B1F,43.063437,141.354229,駐車場なし,日曜日,0,0,0,0,0,0,1


,pref,parking,count
0,三重県,情報なし,5
1,三重県,駐車場あり,43
2,三重県,駐車場なし,2
3,京都府,情報なし,2
4,京都府,駐車場あり,33


,kind,name,point,address,latitude,longitude,parking,holiday,mon_holiday,tue_holiday,wed_holiday,thu_holiday,fri_holiday,sat_holiday,sun_holiday,pref
0,ラーメン,麺屋吉左右,99.746,〒135-0016 東京都江東区東陽1-11-3,35.668468,139.810308,駐車場あり,水曜日・金曜日・日曜日,0,0,1,0,1,0,1,東京都
1,ラーメン,らぁ麺 飯田商店 湯河原本店,99.704,〒259-0303 神奈川県足柄下郡湯河原町土肥2丁目12-14,35.144696,139.109738,駐車場あり,火・水曜日\n※不定期月曜臨時休業あり。,1,1,1,0,0,0,0,神奈川県
2,ラーメン,ラーメン二郎 ひばりヶ丘駅前店,99.400,〒188-0001 東京都西東京市谷戸町3-27-24 ひばりヶ丘プラザ1F,35.749957,139.543549,駐車場あり,水曜夜の部\n日曜\n※祝日は不定休、営業有無は店内表示やTwitterで告知有り,0,0,1,0,0,0,1,東京都


,kind,pref,count
0,つけ麺,京都府,1
1,つけ麺,千葉県,8
2,つけ麺,埼玉県,18


## 種類ごとの分類でランクインした店舗に対してフラグを追加.  
旧型の種類ごとのランキングのデータフレームに対して用いる

## スタイルシートの作成.  

In [3]:
# bootstrapスタイルシートの読み込み
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

## checkbox要素の準備

In [4]:
# optionに設定される要素の準備
vars_parking = [park for park in np.unique(df_pref['parking'].values)]
week_dict = {
  '月':'mon_holiday',
  '火':'tue_holiday',
  '水':'wed_holiday',
  '木':'thu_holiday',
  '金':'fri_holiday',
  '土':'sat_holiday',
  '日':'sun_holiday'
}
vars_pref = ['']
# vars_pref = np.append(vars_pref,[pref for pref in np.unique(df_pref_of_parking['pref'].values)])
vars_pref = [
    "北海道", "青森県", "岩手県", "宮城県", "秋田県", 
    "山形県", "福島県", "茨城県", "栃木県", "群馬県", 
    "埼玉県", "千葉県", "東京都", "神奈川県", "新潟県", 
    "富山県", "石川県", "福井県", "山梨県", "長野県", 
    "岐阜県", "静岡県", "愛知県", "三重県", "滋賀県", 
    "京都府", "大阪府", "兵庫県", "奈良県", "和歌山県", 
    "鳥取県", "島根県", "岡山県", "広島県", "山口県", 
    "徳島県", "香川県", "愛媛県", "高知県", "福岡県", 
    "佐賀県", "長崎県", "熊本県", "大分県", "宮崎県", 
    "鹿児島県", "沖縄県"
]

vars_kind = [kind for kind in np.unique(df_kind['kind'].values)]


## タイトルのコーディング

In [5]:
# タイトル部分のコーディング
title = html.Div(
  [
    dbc.Row(
      html.H2('名店!ラーメンマップ', 
              className='text-white font-italic',
              style={'fontWeight': 'bold', 'fontSize': '24px'}
              ),
      justify="center"  # この属性を使って中央揃え
    )
  ],
  style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'}
)

## sidebar_1のコーディング

In [6]:
# サイドバーのレイアウト定義
sidebar_1 = html.Div(
  [
    # 1枚目のマップに対応したスライダー部分
    dbc.Row(
      [
        # インプット用のコンポーネント
        html.Div(
          [
            # 都道府県のドロップダウンを表示
            html.H5('都道府県を選んでね', style={'margin-top': '8px', 'margin-bottom': '4px'}, className='font-weight-bold'),
            dcc.Dropdown(
              id='select_pref',
              # 表示する要素辞書形式で指定
              options=[{'label': value, 'value': value} for value in vars_pref], 
              # 初期で指定する要素の指定
              value='',
              style={'width': '120px'}
            ),
            html.Br(),
            # 駐車場の有無,空いている店舗を選択できる
            html.H5('駐車場の有無を選んでね', style={'margin-top': '8px', 'margin-bottom': '4px'}, className='font-weight-bold'),
            dcc.Checklist(
              id='check_parking',
              # 表示する要素辞書形式で指定
              options=[{'label': x, 'value': x} for x in vars_parking], 
              # 初期で指定する要素の指定
              value=['駐車場あり', '駐車場なし', '情報なし'],
              # 要素の縦表示か横表示かの指定
              inline=False,
              style={'width': '320px'}
            ),
            html.Br(),
            html.H5('営業する曜日を選んでね', style={'margin-top': '16px', 'margin-bottom': '4px'}, className='font-weight-bold'),
            dcc.Checklist(
              id='check_holiday',
              options=[{'label': key, 'value': value} for key, value in week_dict.items()], 
              value=[], 
              inline=True,
              style={'width': '120px'}
            ),
            html.Button(id='apply_button',
                        n_clicks=0,
                        children='apply',
                        style={'margin-top': '16px'},
                        className='bg-dark text-white'
            ),
          ]
        )
      ],
      style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'}
    )
  ]
)

## sidebar_2のコーディング

In [7]:
# サイドバーのレイアウト定義
sidebar_2 = html.Div(
  [
    # 1枚目のマップに対応したスライダー部分
    dbc.Row(
      [
        # インプット用のコンポーネント
        html.Div(
          [
            # 種類ごとにTOP50の名店のみ表示されるボタンを表示
            html.H5('ジャンルを選んでね', style={'margin-top': '8px', 'margin-bottom': '4px'}, className='font-weight-bold'),
            dcc.Dropdown(
              id='selected_kind',
              # 表示する要素辞書形式で指定
              options=[{'label': value, 'value': value} for value in vars_kind], 
              # 初期で指定する要素の指定
              value='ラーメン',
              style={'width': '120px'}
            ),
            html.Button(id='apply_button_rank',
                        n_clicks=0,
                        children='apply_rank',
                        style={'margin-top': '16px'},
                        className='bg-dark text-white'
            )
          ]
        )
      ],
      style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'}
    )
  ]
)

## コンテンツ箇所のマップの描画と店舗数の定義(初期に使用される)

一列目画像

In [8]:
# hover時に表示するレコードを指定
cols = {
  'rank': True,
  'name': False,
  'point': True,
  'pref': True,
  'address': False,
  'latitude': False,
  'longitude': False,
  'parking': True,
  'holiday': True,
  'mon_holiday': False,
  'tue_holiday': False,
  'wed_holiday': False,
  'thu_holiday': False,
  'fri_holiday': False,
  'sat_holiday': False,
  'sun_holiday': False
}

# 点数順に並び替えてランキングカラムを追加する
df_pref_rank = df_pref.sort_values('point', ascending=False).reset_index(drop=True).reset_index().rename(columns={'index':'rank'})
# 新しく作成したランクカラムを順位になるよう1を足して補正する
df_pref_rank['rank'] = df_pref_rank['rank'].apply(lambda x : int(x)+1)

# scatter_mapboxグラフの作成
fig = px.scatter_mapbox(
  # データフレームと、緯度、軽度の指定
  df_pref_rank, 
  lat="latitude", 
  lon="longitude", 
  # ホバー時の表示設定
  hover_name="name",
  hover_data=cols,
  color_discrete_sequence=["#636EFA"], 
  opacity=0.8,
  #詳細設定
  zoom=3
)
# mapの種類を指定
fig.update_layout(mapbox_style="carto-positron")
# 余白の調整
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

store_num = df_pref.shape[0]

# 選択した都道府県の駐車場情報の比率を円グラフで可視化
df_pref_of_parking_new = df_pref_of_parking.groupby('parking').sum().reset_index()
df_pref_of_parking_new['pref'] = '全国'

fig_pie = px.pie(
  df_pref_of_parking_new,
  values='count', 
  names='parking'
)

fig_pie.update_traces(textposition='inside', textinfo='value')

# レイアウトを設定して背景を透明にする
fig_pie.update_layout(
  paper_bgcolor='rgba(0,0,0,0)',  # 外側の背景色
  plot_bgcolor='rgba(0,0,0,0)',  # グラフ内の背景色
  margin=dict(l=0, r=0, t=0, b=0)  # 余白を削除
)

2枚目の画像

In [9]:
# hover時に表示するレコードを指定
cols_rank = {
  'rank': True,
  'kind': False,
  'name': False,
  'point': True,
  'address': False,
  'latitude': False,
  'longitude': False,
  'parking': True,
  'holiday': True,
  'mon_holiday': False,
  'tue_holiday': False,
  'wed_holiday': False,
  'thu_holiday': False,
  'fri_holiday': False,
  'sat_holiday': False,
  'sun_holiday': False
}

# ラーメン要素でデータフレームの絞り込みを行う
df_kind_of_rank = df_kind.loc[df_kind['kind'] == 'ラーメン'].reset_index(drop=True).reset_index().rename(columns={'index':'rank'})
# 新しく作成したランクカラムを順位になるよう1を足して補正する
df_kind_of_rank['rank'] = df_kind_of_rank['rank'].apply(lambda x : int(x)+1)

# scatter_mapboxグラフの作成
fig_rank = px.scatter_mapbox(
  # データフレームと、緯度、軽度の指定
  df_kind_of_rank.loc[df_kind['kind'] == 'ラーメン'], 
  lat="latitude", 
  lon="longitude", 
  # ホバー時の表示設定
  hover_name="name",
  # name要素以外に表示したいデータの追加
  hover_data=cols_rank,
  color_discrete_sequence=["#6363FA"], 
  opacity=0.8,
  #詳細設定
  zoom=3
)
# mapの種類を指定
fig_rank.update_layout(mapbox_style="carto-positron")
# 余白の調整
fig_rank.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

#対象都道府県の宣言文
kind_prefs =  '都道府県の割合'

fig_pie_pref = px.pie(
  df_kind_of_pref.loc[df_kind_of_pref['kind'] == 'ラーメン'], 
  values='count', 
  names='pref'
)
fig_pie_pref.update_traces(textposition='inside', textinfo='value')

# レイアウトを設定して背景を透明にする
fig_pie_pref.update_layout(
  paper_bgcolor='rgba(0,0,0,0)',  # 外側の背景色
  plot_bgcolor='rgba(0,0,0,0)',  # グラフ内の背景色
  margin=dict(l=0, r=0, t=0, b=0)  # 余白を削除
)

## コールバックの設定

一列目のマップのコールバック関数

In [10]:
@app.callback(
  Output(component_id='store_pref', component_property='children'),
  Output(component_id='ramen_map', component_property='figure'),
  Output(component_id='store_num', component_property='children'),
  Output(component_id='pref_park', component_property='children'),
  Output(component_id='parking_pie', component_property='figure'),
  Input(component_id='apply_button', component_property='n_clicks'),
  State(component_id='check_parking', component_property='value'),
  State(component_id='check_holiday', component_property='value'),
  State(component_id='select_pref', component_property='value')
)
def update_map(n_clicks, selected_parking, selected_holidays, selected_pref):

  if len(selected_pref):
    # 入力された都道府県のデータのみ取得
    df_pref_new = df_pref.query(f"pref == '{selected_pref}'")
  else :
    selected_pref = '全国'
    # 空白のセルが表示された場合全ての店舗を選択する
    df_pref_new = df_pref.copy()

  #selected_parking, selected_holidaysはリスト形式で返却される
  #駐車場に関して指定した要素で、データフレームの絞り込みを行う
  # パーキング要素が指定されていれば発動
  if len(selected_parking):
    df_pref_new = df_pref_new[df_pref_new['parking'].isin(selected_parking)]
  
  if len(selected_holidays):
    for col in selected_holidays:
      # 定休日でない店舗を表示する
      # 渡されたカラムが0であるインデックスのみ表示
      df_pref_new = df_pref_new[df_pref_new[col] == 0]

  # 店舗が存在しない場合の策も考える
  if df_pref_new.empty:
    # ユーザーに表示するメッセージ
    no_data_message = "該当する店舗はありません…"
    # 画像ファイルの読み込み
    img = Image.open("../image/sorry.png")
    # NumPy配列に変換
    img_array = np.array(img)
    # NumPy配列に変換した画像データをPlotlyのImageトレースに渡してグラフを作成
    # Imageトレースの作成
    trace = go.Image(z=img_array)
    # フィギュアの作成
    fig = go.Figure(trace)

    # ホバー時の情報表示を無効化
    fig.update_traces(hoverinfo='none')
    # レイアウトの調整（必要に応じて）
    fig.update_layout(
      margin=dict(l=0, r=0, t=0, b=0),
      xaxis=dict(showgrid=False, showticklabels=False),
      yaxis=dict(showgrid=False, showticklabels=False)
    )

    # 他の出力にも適切な値を設定
    return no_data_message, fig, no_data_message, no_data_message, fig
      
  # 条件に従う情報をもとに、駐車場の比率を表示するためのデータフレームを生成する
  # 都道府県、駐車場の属性ごとにグループ化、シリーズの作成
  se_pref_eda_of_holiday = df_pref_new[['pref','parking']].groupby(['pref','parking']).size()
  # データフレーム化。それぞれをcolumnsになるようインデックスをリセット
  df_pref_of_parking_new = pd.DataFrame(se_pref_eda_of_holiday, columns=['count']).reset_index()
  

  # 点数順に並び替えてランキングカラムを追加する
  # 一度目のreset_index(drop=True)では新規のインデックスが追加されない（インデックスがリセットもされない）
  df_pref_new = df_pref_new.sort_values('point', ascending=False).reset_index(drop=True).reset_index().rename(columns={'index':'rank'})
  # 新しく作成したランクカラムを順位になるよう1を足して補正する
  df_pref_new['rank'] = df_pref_new['rank'].apply(lambda x : int(x)+1)
  
  # 上記の処理によりデータフレーム完成
  # scatter_mapboxグラフの作成
  fig = px.scatter_mapbox(
    # データフレームと、緯度、軽度の指定
    df_pref_new, 
    lat="latitude", 
    lon="longitude", 
    # ホバー時の表示設定
    hover_name="name",
    hover_data=cols,
    color_discrete_sequence=["#636EFA"], 
    opacity=1,
    #詳細設定
    zoom=7
  )
  # mapの種類を指定
  fig.update_layout(mapbox_style="carto-positron")
  # 余白の調整
  fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

  # 小タイトルの設定
  store_pref = selected_pref + 'の人気店だよ！'

  #対象店舗数のカウント
  store_num = df_pref_new.shape[0]
  store_num = '対象の店舗は' + str(store_num) + 'だよ！'

  #対象都道府県の宣言文
  pref_park = selected_pref + 'の駐車場の有無割合'

  # 選択した都道府県の駐車場情報の比率を円グラフで可視化
  fig_pie = px.pie(
    df_pref_of_parking_new,
    values='count', 
    names='parking'
  )
  fig_pie.update_traces(textposition='inside', textinfo='value')

  # レイアウトを設定して背景を透明にする
  fig_pie.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',  # 外側の背景色
    plot_bgcolor='rgba(0,0,0,0)',  # グラフ内の背景色
    margin=dict(l=0, r=0, t=0, b=0)  # 余白を削除
  )
  
  return store_pref, fig, store_num, pref_park, fig_pie

2枚目の画像のコールバック関数

In [11]:
@app.callback(
  Output(component_id='kind_rank', component_property='children'), 
  Output(component_id='rank_map', component_property='figure'), 
  Output(component_id='rank_of_pref', component_property='figure'),
  Input(component_id='apply_button_rank', component_property='n_clicks'),
  State(component_id='selected_kind', component_property='value')
)
def update_map(n_clicks, selected_kind):
  # データフレームの作成を行う
  # 選択された種類の要素でデータフレームの絞り込みを行う
  # 一度目のreset_index(drop=True)では新規のインデックスが追加されない（インデックスがリセットもされない）
  df_kind_of_rank = df_kind.loc[df_kind['kind'] == selected_kind].reset_index(drop=True).reset_index().rename(columns={'index':'rank'})
  # 新しく作成したランクカラムを順位になるよう1を足して補正する
  df_kind_of_rank['rank'] = df_kind_of_rank['rank'].apply(lambda x : int(x)+1)
  # 選択された種類の要素で絞り込み
  df_kind_of_pref_new = df_kind_of_pref.loc[df_kind_of_pref['kind'] == selected_kind]
  
  # 上記の処理によりデータフレーム完成
  # scatter_mapboxグラフの作成
  fig_rank = px.scatter_mapbox(
    # データフレームと、緯度、軽度の指定
    data_frame = df_kind_of_rank, 
    lat="latitude", 
    lon="longitude", 
    # ホバー時の表示設定
    hover_name="name",
    hover_data=cols_rank,
    color_discrete_sequence=["#636EFA"], 
    opacity=1,
    #詳細設定
    zoom=7
  )
  # mapの種類を指定
  fig_rank.update_layout(mapbox_style="carto-positron")
  # 余白の調整
  fig_rank.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

  # 小タイトルの設定
  kind_rank = selected_kind + "百名店"

  fig_pie_pref = px.pie(
    df_kind_of_pref_new.sort_values('pref'),
    values='count', 
    names='pref'
  )
  fig_pie_pref.update_traces(textinfo='value')

  # レイアウトを設定して背景を透明にする
  fig_pie_pref.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',  # 外側の背景色
    plot_bgcolor='rgba(0,0,0,0)',  # グラフ内の背景色
    margin=dict(l=0, r=0, t=0, b=0)  # 余白を削除
  )

  return kind_rank, fig_rank, fig_pie_pref

## コンテンツ箇所の画像の貼り付け

In [12]:
content_1_1 = html.Div(
  [
    dbc.Row(
      [
        # 小タイトルの受け取り
        html.H4(id='store_pref'),
        # dcc.graphコンポーネントを利用して、グラフをプロットする
        dcc.Graph(id='ramen_map', config={'responsive': True}),
        html.P(id='store_num')
      ]
    )
  ]
)
content_1_2 = html.Div(
  [
    dbc.Row(
      [
        # dcc.graphコンポーネントを利用して、グラフをプロットする
        html.P(id='pref_park'),
        dcc.Graph(id='parking_pie')
      ]
    )
  ]
)

content_2_1 = html.Div(
  [
    dbc.Row(
      [
        html.H4(id='kind_rank'),
        # dcc.graphコンポーネントを利用して、グラフをプロットする
        dcc.Graph(id='rank_map', config={'responsive': True})
      ]
    )
  ]
)
content_2_2 = html.Div(
  [
    dbc.Row(
      [
        html.P('都道府県の比率'),
        # dcc.graphコンポーネントを利用して、グラフをプロットする
        dcc.Graph(id='rank_of_pref')
      ]
    )
  ]
)

## アプリの起動

In [13]:
# 画面全体のgridの縦比を、1：3に分割する
app.layout = dbc.Container(
  [
    dbc.Row(
      [
        dbc.Col(title, className='bg-warning')
      ]
    ),
    dbc.Row(
      [
        dbc.Col(sidebar_1, width=3, className='bg-info'),
        dbc.Col(content_1_1, width=5, className='bg-light'),
        dbc.Col(content_1_2, width=4, className='bg-light')
      ]
    ),
    dbc.Row(
      [
        dbc.Col(sidebar_2, width=3, className='bg-warning'),
        dbc.Col(content_2_1, width=5, className='bg-light'),
        dbc.Col(content_2_2, width=4, className='bg-light')
      ]
    ),    
  ],
  # 不要な余白の除去
  fluid=True
)

if __name__ == "__main__":
  app.run_server(debug=True, port=1235)
